# Vector DB

In this notebook, we use a containerized version of Chroma DB. To set up, you will need the following:

1. Install [Docker Desktop](https://www.docker.com/products/docker-desktop/) by following the link and Download Docker Desktop for your operating system.
2. In a terminal window, navigate to the folder ./05_src/chromadb/. For example, on Windows, you would use `cd .\05_src\chromadb`.
3. Run the command `docker compose up -d`, which will start the Chroma DB server.

## Downloading Batch Results

In the previous notebook, we had created batch processes. We will start by consulting the status of our batch processes by identifying them throught their descriptions.

In [ ]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [ ]:
batch_description = 'Pitchfork reviews content embeddings 2025-10-18 12:17:17'

In [ ]:
from openai import OpenAI

client = OpenAI()

batch_processes = client.batches.list().to_dict()
batch_info= [
    {'batch_id': batch['id'],
     'description': batch['metadata']['description'],
    'status': batch['status'],
    'request_counts': batch['request_counts'],
    'output_file_id': batch['output_file_id'],
    'input_file_id': batch['input_file_id']}
            for batch in batch_processes['data'] if batch['metadata']['description'] == batch_description
    ]
batch_info

When the status of the batches is complete, we can query the `output_file_id` where their results will be stored.

More generally, we will require the original text and the embeddings of that original text mapped through the `custom_id`.

In [ ]:
batch_complete = [
    batch  for batch in batch_info if batch['status'] == 'completed'
]
batch_complete

Before we download all results, examine the response of the file API:

In [ ]:
response = client.files.content(batch_complete[0]['output_file_id'])
text_response = response.text
lines = text_response.split('\n')
print(lines[0])


For our results database, we will need to map the original text to their embeddings. 

In [ ]:
import json 

def get_text_and_embeddings(batch):
    embedding_lines =  get_content_from_file(batch, 'output_file_id')
    text_lines = get_content_from_file(batch, 'input_file_id')
    return embedding_lines, text_lines

def get_content_from_file(batch, key):
    file = client.files.content(batch[key])
    text = file.text
    lines = text.split('\n')
    content_lines = [json.loads(line) for line in lines if line.strip()]
    return content_lines


Notice that the response is also a .jsonl file. Therefore, we can process it line-by-line and use the `custom_id` to map to the original document chunk.

The function below:

- Creates a dictionary, `text_dict`, with keys given by each `custom_id` and value equal to the text.
- Iterate over all embedding items and use the dictionary defined above to map the embeddings to their input text.

In [ ]:
def create_chroma_inputs(embedding_lines, text_lines):
    chroma_inputs = []
    text_dict = {item['custom_id']: item['body']['input'] for item in text_lines}
    for embed_item in embedding_lines:
        custom_id = embed_item['custom_id']
        text = text_dict.get(custom_id, "")
        chroma_input = {
            'id': embed_item['custom_id'],
            'embedding': embed_item['response']['body']['data'][0]['embedding'],
            'text': text
        }
        chroma_inputs.append(chroma_input)
    return chroma_inputs

A couple of functions to control the logic flow:

In [ ]:
from tqdm import tqdm

def process_batch_for_chromadb(batch):
    embedding_lines, text_lines = get_text_and_embeddings(batch)
    chroma_inputs = create_chroma_inputs(embedding_lines, text_lines)
    return chroma_inputs

def process_batches_for_chromadb(batches):
    all_chroma_inputs = []
    for batch in tqdm(batches, desc="Processing batches"):
        chroma_inputs = process_batch_for_chromadb(batch)
        all_chroma_inputs.extend(chroma_inputs)
    return all_chroma_inputs

Now, we can create our input dictionaries.

In [ ]:
chroma_inputs = process_batches_for_chromadb(batch_complete)

In [ ]:
chroma_inputs[1]

# Load Embeddings to Chroma

In [ ]:

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os


def setup_collection(chroma_url:str="http://localhost:8000",
                     collection_name: str = "pitchfork_reviews"):
    chroma_client = chromadb.HttpClient(host=chroma_url)
    collections = chroma_client.list_collections()
    if collection_name in [col.name for col in collections]:
        chroma_client.delete_collection(name=collection_name)

    collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=OpenAIEmbeddingFunction(
            api_key = os.getenv("OPENAI_API_KEY"),
            model_name="text-embedding-3-small")
        )
    return collection

def load_embeddings_to_db(chroma_inputs:list[dict], 
                          collection_name:str,
                          chroma_url:str="http://localhost:8000",
                          batch_size:int= 1000
                          ):

    
    collection = setup_collection(chroma_url=chroma_url, collection_name=collection_name)

    for i in tqdm(range(0, len(chroma_inputs), batch_size)):
        batch = chroma_inputs[i:i + batch_size]
        collection.add(
            documents=[item['text'] for item in batch],
            embeddings=[item['embedding'] for item in batch],
            ids=[item['id'] for item in batch]
        )


In [ ]:
vector_db_client_url:str="http://localhost:8000"
load_embeddings_to_db(chroma_inputs=chroma_inputs,
                      collection_name="pitchfork_reviews",
                      chroma_url=vector_db_client_url, 
                      batch_size=1000)

# Additional Details

We will use a simple database to store additional details about the reviews. In this case, we load the jsonl files, and use pandas to create a few tables in a sql database. The connection string to the database is included in the .secrets file.

In [ ]:
import json

def load_jsonl(file:str):
    data = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

In [ ]:
import pandas as pd
import sqlalchemy as sa
import os

doc_folder = "../../05_src/documents/"
tables = ["artists", "reviews", "labels", "genres"]

def upload_tables_to_sql(tables:list[str], doc_folder:str):
    engine = sa.create_engine(os.getenv("SQL_URL"))
    for table_name in tables:
        file_path = os.path.join(doc_folder, f"pitchfork_{table_name}.jsonl")
        data = load_jsonl(file_path)
        df = pd.DataFrame(data)
        with engine.connect() as conn:
            df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Loaded {df.shape} records from {file_path}")

upload_tables_to_sql(tables=tables, doc_folder=doc_folder)

In [ ]:
def additional_details(review_id:str):
    import sqlalchemy as sa
    import pandas as pd
    import os

    engine = sa.create_engine(os.getenv("SQL_URL"))
    query = f"""
    SELECT r.reviewid,
		r.title,
		r.artist,
		r.score,
		g.genre
    FROM reviews AS r
    LEFT JOIN genres as g
	    ON r.reviewid = g.reviewid
    WHERE r.reviewid = '{review_id}'
    """
    with engine.connect() as conn:
        result = pd.read_sql(query, conn)
    if not result.empty:
        row = result.iloc[0]
        details = {
            "reviewid": row['reviewid'],
            "album": row['title'],
            "score": row['score'],
            "artist": row['artist']
        }
        return details
    else:
        return {}
    
def get_reviewid_from_custom_id(custom_id:str):
    return custom_id.split('_')[0]

# Prompt Generator

Here we create a prompt with the context gathered through the different data operations.

In [ ]:
chroma = chromadb.HttpClient(host=vector_db_client_url)
collection = chroma.get_collection(name="pitchfork_reviews", 
                                   embedding_function=OpenAIEmbeddingFunction(
                                       api_key = os.getenv("OPENAI_API_KEY"),
                                       model_name="text-embedding-3-small")
                                   )


In [ ]:
collection.query(
    query_texts=["A great album with stunning vocals and production."],
    n_results=3
)

In [ ]:
def get_context_data(query:str, collection:chromadb.api.models.Collection, top_n:int):
    results = collection.query(
        query_texts=[query],
        n_results=top_n
    )
    context_data = []
    for idx, custom_id in enumerate(results['ids'][0]):
        review_id = get_reviewid_from_custom_id(custom_id)
        details = additional_details(review_id)
        details['text'] = results['documents'][0][idx]
        context_data.append(details)
    return context_data

def generate_prompt(query:str, collection:chromadb.api.models.Collection, top_n:int):
    context_data = get_context_data(query, collection, top_n)
    prompt = f"Given a query, provide a detailed response using the context from relevant Pitchfork reviews. The context will contain references to {top_n} album reviews.\n\n"
    prompt += f"The score is numeric and its scale is from 0 to 10, with 10 being the highest rating. Any album with a score greater than 8.0 is considered a must-listen; album with a score greater than 6.5 is good.\n\n"
    prompt += f"<query>{query}</query>\n\n"
    prompt += "<context>\n"
    for k, context in enumerate(context_data):
        prompt += f"<album {k}>\n"
        prompt += f"- Album Title: {context.get('album', 'N/A')}\n" 
        prompt += f"- Album Artist: {context.get('artist', 'N/A')}\n"
        prompt += f"- Album Score: {context.get('score', 'N/A')}\n"
        prompt += f"- Review Quote: {context.get('text', 'N/A')}\n"
        prompt += f"</album {k}>\n\n"
    prompt += "</context>\n\n"
    prompt += "\nBased on the context and nothing else, provide a detailed response to the query."
    return prompt

def generate_response(query:str, collection:chromadb.api.models.Collection, top_n:int=1):
    prompt = generate_prompt(query, collection, top_n)
    print("Generated Prompt:\n", prompt)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides information based on Pitchfork reviews."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.7
    )
    return response.choices[0].message.content

# Query

We can now use chroma's similarity function to query the database. Notice that the query itself needs to be converted to embeddings, so we must provide an `embedding_function`. In this case, we use `OpenAIEmbeddingFunction()` to get compatible embeddings using model `text-embedding-3-small`.

In [ ]:
response = generate_response("What are some highly rated albums by emerging indie artists?", collection, 3)

In [ ]:
print(response)

**Note**: Try changing the top_n parameter to 1 and re-run the query. 